<a href="https://colab.research.google.com/github/poovarasansivakumar2003/Marvel_works/blob/main/Task_1_Decision_Tree_based_ID3_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Decision Trees: An Overview

#### Decision Trees and Terminology

A decision tree is a powerful tool used in machine learning for classification and regression tasks. It mimics the human decision-making process by breaking down data into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes.

- **Root Node**: The top node of a decision tree that represents the entire dataset, which is then split into subsets.
- **Decision Node**: A node that splits into further sub-nodes based on certain conditions.
- **Leaf Node (Terminal Node)**: Nodes that do not split further, representing the final decision or classification.
- **Branches**: Links between nodes that represent the decision paths taken.

#### Entropy and Information Gain

**Entropy** measures the amount of uncertainty or impurity in a dataset. It is given by the formula:

![Entropy](https://miro.medium.com/v2/resize:fit:640/format:webp/1*voRgqFZhfko4ZfG3odSK_Q.png)

where pi is the proportion of each category i=1,…,N.

**Information Gain** quantifies the reduction in entropy achieved by partitioning the dataset according to an attribute. It is the difference between the entropy of the dataset before and after the split:

![IG](https://miro.medium.com/v2/resize:fit:640/format:webp/1*_uzMwgApeQPXEylcwOc3WQ.png)

where j is particular feature.

#### ID3 Algorithm

ID3 (Iterative Dichotomiser 3) is a simple decision tree algorithm introduced by Ross Quinlan in 1986. It uses entropy and information gain to construct a decision tree by recursively partitioning the dataset.

##### Procedure for ID3

1. **Calculate Entropy** of the entire dataset.
2. **For each attribute**, calculate the information gain when the dataset is split by that attribute.
3. **Select the attribute** with the highest information gain as the decision node.
4. **Split the dataset** into subsets based on the chosen attribute.
5. **Repeat recursively** for each subset, using only the remaining attributes.
6. **Stop** when all instances in a subset belong to the same class or no more attributes are available.


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from collections import Counter

class Node:
    def __init__(self, attribute=None, threshold=None, left=None, right=None, *, value=None):
        self.attribute = attribute
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def entropy(y):
    proportions = np.bincount(y) / len(y)
    return -np.sum([p * np.log2(p) for p in proportions if p > 0])

def information_gain(X, y, attribute, threshold):
    parent_entropy = entropy(y)
    left_idx = X[:, attribute] <= threshold
    right_idx = X[:, attribute] > threshold
    if np.sum(left_idx) == 0 or np.sum(right_idx) == 0:
        return 0
    n, n_left, n_right = len(y), np.sum(left_idx), np.sum(right_idx)
    left_entropy, right_entropy = entropy(y[left_idx]), entropy(y[right_idx])
    child_entropy = (n_left / n) * left_entropy + (n_right / n) * right_entropy
    return parent_entropy - child_entropy

def best_split(X, y):
    best_gain = 0
    best_attribute, best_threshold = None, None
    for attribute in range(X.shape[1]):
        thresholds = np.unique(X[:, attribute])
        for threshold in thresholds:
            gain = information_gain(X, y, attribute, threshold)
            if gain > best_gain:
                best_gain, best_attribute, best_threshold = gain, attribute, threshold
    return best_attribute, best_threshold

def build_tree(X, y, depth=0, max_depth=None):
    if len(set(y)) == 1:
        return Node(value=y[0])
    if max_depth is not None and depth >= max_depth:
        return Node(value=Counter(y).most_common(1)[0][0])
    attribute, threshold = best_split(X, y)
    if attribute is None:
        return Node(value=Counter(y).most_common(1)[0][0])
    left_idx = X[:, attribute] <= threshold
    right_idx = X[:, attribute] > threshold
    left = build_tree(X[left_idx], y[left_idx], depth + 1, max_depth)
    right = build_tree(X[right_idx], y[right_idx], depth + 1, max_depth)
    return Node(attribute=attribute, threshold=threshold, left=left, right=right)

def predict_tree(tree, X):
    if tree.value is not None:
        return tree.value
    if X[tree.attribute] <= tree.threshold:
        return predict_tree(tree.left, X)
    else:
        return predict_tree(tree.right, X)

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Build the decision tree
tree = build_tree(X, y, max_depth=3)

# Predict on the dataset
predictions = [predict_tree(tree, x) for x in X]

# Calculate accuracy
accuracy = np.sum(predictions == y) / len(y)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 97.33%
